In [ ]:
!pip install -q cassio datasets langchain openai tiktoken

In [ ]:
!pip install -U langchain-community

Import the packges needed

In [ ]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

from datasets import load_dataset

import cassio

In [6]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.4 MB/s eta 0:00:00


In [7]:
from PyPDF2 import PdfReader

Setup

In [ ]:
ASTRA_DB_APPLICATION_TOKEN="Astra"
ASTRA_DB_ID="Astra"

OPENAI_API_KEY = ""

In [ ]:
pdfreader = PdfReader(" ")

In [ ]:
from typing_extensions import Concatenate
# read the text from pdf

raw_text = ""
for i, page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    raw_text += content

In [ ]:
raw_text

Initiialize the connection too your database

In [ ]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)


Create the Langchain embedding and LLM objects for the later usage

In [ ]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

Create your Langchain vector store... backed by Astra DB

In [ ]:
astra_vector_store = Cassandra(
    embedding = embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None
)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Charaacter Text Split such that it will nit increase token size
text_splitter = CharacterTextSplitter(
    seperator= "\n",
    chunk_size = 800,
    chunk_overlap= 200,
    length_function= len,
)
texts = text_splitter.split_text(raw_text)



In [ ]:
texts[:50]

Load the dataset into the vector store

In [ ]:
astra_vector_store.add_texts(texts[:50])

print("Inserted %i headlines." % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore = astra_vector_store)

Run QA Cycle

In [ ]:
first_question = True
while True:
  if first_question:
    query_text = input("\nEnter your question (or type 'quit')")
  else:
    query_text = input("\nWhat's your next question (or type 'quit')")

  if query_text.lower() == "quit":
    break
  if query_text == "":
    continue

  first_question = False

  print("\nQuestion: \"%s\"" % query_text)
  answer = astra_vector_index.query(query_text, llm=llm).strip()
  print("ANSWER: \"%s\"" % answer)

  print("FIRST DOUMENTS BY RELEVANCE:")
  for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4)
    print(" [%0.4f] \"%s ...\"" % (score, doc.page_conteny[:84]))
